<a href="https://colab.research.google.com/github/iypc-team/CoLab/blob/master/Writefile_GetTPU_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import absolute_import
import os, shutil
import tensorflow as tf

try: shutil.rmtree('/content/sample_data')
except: pass
print('Updated: 10/14/2022-7')

In [ ]:
%%writefile GetTPU2.py
# Updated: 10/14/2022

from __future__ import absolute_import
from google.colab import drive
import glob, os, re, time, shutil
from os.path import abspath, basename, exists, join
from pathlib import Path

import tensorflow as tf
import numpy as np
np.set_printoptions(4)
from matplotlib import pyplot as plt

programStart=time.time()
contentPath=os.getcwd()

bullshitPath=join(contentPath, 'sample_data')
if exists(bullshitPath):
    shutil.rmtree(bullshitPath)
    time.sleep(5)
    print(bullshitPath, 'removed')

drive.mount('/content/drive', force_remount=True)

os.chdir('/content/drive/MyDrive/PythonFiles')
# print(f'cwd: {Path.cwd()}')
# %ls -l

os.chdir(contentPath)

impPath = '/content/drive/MyDrive/BashColors.py'
if not exists('BashColors.py'):
    print(contentPath)
    print(impPath)
    shutil.copy2(impPath, contentPath)
    time.sleep(1)
    from BashColors import C
else:
    from BashColors import C
    print(f'{os.path.basename(impPath)}{C.BGreen} exists{C.ColorOff}')

AUTO = tf.data.experimental.AUTOTUNE

class GetTPU:
    """with strategy.scope():"""
    gt = None
    tpu = None
    strategy = None

    def __init__(self):
        self.updated='Updated: 10/14/2022'
        self.AUTO = tf.data.experimental.AUTOTUNE
        self.tpu = None
        self.strategy = None
        
    # def __all__(self) = [GetTPU, tpu, strategy, gt]

    def getTPU(self):
        return self.tpu

    def getStrategy(self):
        return self.strategy

    def connectTPU(self):
        # TPUClusterResolver automatically checks connected TPU on all Google platforms
        try:
            self.tpu = None
            # tpu detection
            self.tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
            tf.config.experimental_connect_to_cluster(self.tpu)
            tf.tpu.experimental.initialize_tpu_system(self.tpu)
            self.strategy = tf.distribute.TPUStrategy(self.tpu)
            with self.strategy.scope():
                print(f'{C.BGreen}Number of TPUs: {gt.strategy.num_replicas_in_sync}{C.ColorOff}')
                # print('using TPU')

        except ValueError as err:
            # print(err)
            if tf.config.list_physical_devices('GPU'):
                # print(tf.config.list_physical_devices(), '\n')
                try:
                    self.strategy = tf.distribute.MirroredStrategy() # for GPU or multi-GPU machines
                    with strategy.scope():
                        print('using GPU')
                except:
                    self.strategy = tf.distribute.get_strategy()# default strategy for CPU
                    with strategy.scope():
                        print('using CPU')
            
            # for clusters of multi-GPU machines
            # self.strategy=tf.distribute.experimental.MultiWorkerMirroredStrategy()

gt = GetTPU()
gt.connectTPU()
tpu = gt.getTPU()
strategy = gt.getStrategy()
print(f'{gt.updated}')

In [ ]:
from GetTPU2 import *

In [ ]:
q

In [ ]:
def getTPUInfo():
    from tensorflow.python.profiler import profiler_client
    tpu_profile_service_address = os.environ['COLAB_TPU_ADDR'].replace('8470', '8466')
    print(profiler_client.monitor(tpu_profile_service_address, 100, 2))
    tpu_profile_service_address = os.environ['COLAB_TPU_ADDR'].replace('8466', '8470')


getTPUInfo()

In [ ]:
import tensorflow as tf
# print(tf.config.list_physical_devices())

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    # print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    tpu_strategy = tf.distribute.TPUStrategy()
    with tpu_strategy:
        print(tf.config.list_physical_devices)

except BaseException as err:
    print(err)

if tf.config.list_physical_devices('GPU'):
    print(tf.config.list_physical_devices('GPU'))

if tf.config.list_physical_devices('TPU'):
    print(tf.config.list_physical_devices())
    # tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection


In [ ]:
import tensorflow as tf
if tf.config.list_physical_devices('GPU'):
    print(tf.config.list_physical_devices('GPU'))

else:
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
        print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        tpu_strategy = tf.distribute.TPUStrategy()
    except Exception as err:
        print('err:', err)

In [ ]:
import tensorflow as tf
import timeit

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()

if device_name != '/device:GPU:0':
    print('GPU device not found')
else: 
    print('Found GPU at: {}'.format(device_name))

In [ ]:
import tensorflow as tf

if len(tf.config.list_physical_devices()) == 1:
    print('using cpu')
    
    strategy = tf.distribute.get_strategy()
    with strategy.scope():
        print(strategy)
elif len(tf.config.list_physical_devices()) == 2:
    print('using gpu')
    
    strategy = tf.distribute.MirroredStrategy()
    print(strategy)
else:
    print('tpu')


In [ ]:
import tensorflow as tf
strategy = None
if tf.config.list_physical_devices('GPU'):
    print(tf.config.list_physical_devices(), '\n')
    strategy = tf.distribute.MirroredStrategy()

elif tf.config.list_physical_devices('TPU'):
    print(tf.config.list_physical_devices(), '\n')
    strategy = tf.distribute.TPUStrategy()

elif tf.config.list_physical_devices('CPU'):
    print(tf.config.list_physical_devices(), '\n')
    strategy = tf.distribute.get_strategy()

with strategy.scope():
  # Do something interesting
    print(tf.Variable(1.))

In [ ]:
dir(tf.test)

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name() # this will tell you device number (should be 0 with a single GPU)
tf.test.gpu_device_name()       # this will tell you device number (should be 0 with a single GPU)

In [ ]:
import tensorflow as tf
device = tf.config.get_visible_devices()
print(len(device))
print(device)
for dev in device:
    print(dev)
# dir(tf.config)

In [ ]:
import tensorflow as tf
try:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    print("All devices: ", tf.config.list_logical_devices('TPU'))
    strategy = tf.distribute.TPUStrategy(resolver)
except ValueError:
    strategy = tf.distribute.get_strategy()
    print(strategy)

In [ ]:
dir(tf.distribute)

In [ ]:
import tensorflow as tf
from tensorflow.python.platform.tf_logging import log
logicalDevices = tf.config.list_logical_devices()
print(type(logicalDevices))
print(len(logicalDevices))
for dev in logicalDevices:
    print(dev)

In [ ]:
bs="""
import tensorflow as tf
try:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    print("All devices: ", tf.config.list_logical_devices('TPU'))
    strategy = tf.distribute.TPUStrategy(resolver)
except ValueError:
    strategy = tf.distribute.get_strategy()
"""
print(bs)